In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from helpers import *
from implementations import *
import pandas as pd

In [2]:
DATA_TRAIN_PATH = 'train.csv'
y_or, tX_or, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
y = y_or
tX = tX_or

In [52]:
tX = pd.DataFrame(tX)
tX.where(tX!=-999, inplace=True)
tX.fillna(tX.median(), inplace=True)    
tX = tX.values

In [4]:
y = y_for_logistic(y)

In [5]:
x_tr_tot, x_te_tot, y_tr_tot, y_te_tot = split_data(y,tX,0.8,1)

In [55]:
initial_w = np.zeros(x_tr_tot.shape[1])
max_iters = 10000
gamma = 1e-10
w, loss = logistic_regression(y_tr_tot, x_tr_tot, initial_w, max_iters, gamma)

Current iteration=0, loss=138629.43611198905
Current iteration=1000, loss=106751.60151235267
Current iteration=2000, loss=106408.02717390667
Current iteration=3000, loss=106299.1933633236
Current iteration=4000, loss=106253.50135402067
Current iteration=5000, loss=106230.50607151032
Current iteration=6000, loss=106216.27284014336
Current iteration=7000, loss=106205.50922362538
Current iteration=8000, loss=106196.14474365817
Current iteration=9000, loss=106187.35953373656


In [58]:
y_pred = predict_labels(w, x_te_tot, True)
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7362


In [59]:
DATA_TEST_PATH = 'test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [60]:
#cell for logistic
y_pred_test = predict_labels(w, tX_test, True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]

In [61]:
initial_w = np.zeros(x_tr_tot.shape[1])
lambda_ = 1e-8
max_iters = 10000
gamma = 1e-10
w, loss = reg_logistic_regression(y_tr_tot, x_tr_tot, lambda_, initial_w, max_iters, gamma)

Current iteration=0, loss=134001.19392646643
Current iteration=1000, loss=106750.92508967363
Current iteration=2000, loss=106407.85577669005
Current iteration=3000, loss=106299.12698797951
Current iteration=4000, loss=106253.47056667147
Current iteration=5000, loss=106230.48878656604
Current iteration=6000, loss=106216.26085869093
Current iteration=7000, loss=106205.49936369108
Current iteration=8000, loss=106196.13575003046
Current iteration=9000, loss=106187.3509072507


In [62]:
y_pred = predict_labels(w, x_te_tot, True)
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7362


In [63]:
#cell for logistic
y_pred_test = predict_labels(w, tX_test, True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]

In [7]:
def cross_validation(y, x, k_indices, k, lambda_, gamma, acc = False, ls = False):
    """return the loss of ridge regression."""
    losses_tr = []
    losses_te = []
    accuracies = []
    ws = []
    initial_w = np.zeros(x.shape[1])

    
    for k_group in range(k):
        index_te = k_indices[k_group]
        index_tr = np.setdiff1d(np.arange(len(y)), index_te)
        #print(index_te, index_tr)
        x_te = x[index_te]
        x_tr = x[index_tr]
        y_te = y[index_te]
        y_tr = y[index_tr]
        
        # form data with polynomial degree
        x_te_poly = x_te
        x_tr_poly = x_tr
        
        if (ls==False):
            w, _ = reg_logistic_regression(y_tr, x_tr_poly, lambda_, initial_w, 1000, gamma)
        else:
            w = least_squares_lstsq_ver(y_tr, x_tr_poly)
        
        # calculate the loss for train and test data
        rmse_tr = calculate_loss(y_tr, x_tr_poly, w)
        rmse_te = calculate_loss(y_te, x_te_poly, w)
        #print(lambda_, rmse_te)
        losses_tr.append(rmse_tr)
        losses_te.append(rmse_te)
        
        y_pred = predict_labels(w, x_te_poly)
        accuracies.append(accuracy(y_te, y_pred))
        
        ws.append(w)
        
    if acc==False:
        loss_tr = np.median(losses_tr)
        loss_te = np.median(losses_te)
        return loss_tr, loss_te, np.mean(ws, axis=0)
    else:
        return np.mean(accuracies), np.mean(ws, axis=0)

In [25]:
def select_best_hypers(y, x, k_fold, seed=1):
    #y_sub, x_sub = get_subsample(y, x, 100000, seed)
    lambdas = np.logspace(-10, 0, 30)
    lambdas = [1e-10]
    gammas = np.logspace(-10, 0, 30)
    #lambdas=[1e-06]
    k_indices = build_k_indices(y, k_fold, seed)
    loss_min = np.inf
    gamma_star = 0
    lambda_star = 0
    w_star = 0
    for gamma in gammas:
        for lambda_ in lambdas:
            #print(degree)
            loss_tr, loss_te, w = cross_validation(y, x, k_indices, k_fold, lambda_, gamma)
            #print(loss_te)
            if loss_te < loss_min:
                loss_min = loss_te
                print("New loss: {}, gamma: {}, lambda: {}".format(loss_te, gamma, lambda_))
                gamma_star = gamma
                lambda_star = lambda_
                w_star = w
    return gamma_star, lambda_star, loss_min, w_star

In [26]:
k_fold = 8
gammas_star=[]
lambdas_star=[]
w_star = []
for jet in range(0, 8):
    print("jet {}: ".format(jet))
    gamma_star, lambda_star, loss, w = select_best_hypers(y_tr[jet], tX_tr[jet], k_fold, 1)
    gammas_star.append(gamma_star)
    lambdas_star.append(lambda_star)
    w_star.append(w)
    #print("jet {}: Best accuracy {}, degree: {},  lambda: {}".format(jet, acc, degree_star, lambda_star))
    print("jet {}: best loss: {}, degree: {}, lambda: {}".format(jet, loss, gamma_star, lambda_star))

jet 0: 
New loss: 1795.6522157325862, gamma: 1e-10, lambda: 1e-10
New loss: 1781.2144753402767, gamma: 2.2122162910704502e-10, lambda: 1e-10
New loss: 1750.0401072214504, gamma: 4.893900918477499e-10, lambda: 1e-10
New loss: 1684.6675541796776, gamma: 1.0826367338740564e-09, lambda: 1e-10
New loss: 1556.065366850918, gamma: 2.395026619987491e-09, lambda: 1e-10
New loss: 1335.1674854273551, gamma: 5.298316906283702e-09, lambda: 1e-10
New loss: 1043.929227467509, gamma: 1.1721022975334793e-08, lambda: 1e-10
New loss: 793.712284585019, gamma: 2.592943797404667e-08, lambda: 1e-10
New loss: 668.980413708281, gamma: 5.736152510448681e-08, lambda: 1e-10
New loss: 635.9759355542362, gamma: 1.2689610031679235e-07, lambda: 1e-10
New loss: 630.12605180581, gamma: 2.807216203941181e-07, lambda: 1e-10
New loss: 623.4875319017765, gamma: 6.210169418915616e-07, lambda: 1e-10
New loss: 610.2879758679297, gamma: 1.3738237958832638e-06, lambda: 1e-10
New loss: 586.2374445964485, gamma: 3.039195382313201

D:\EPFL\ML\github_mine\ML\Project_1\final_zip\implementation_helpers.py:102: RuntimeWarning: overflow encountered in log1p
  """
D:\EPFL\ML\github_mine\ML\Project_1\final_zip\implementation_helpers.py:102: RuntimeWarning: invalid value encountered in log1p
  """


jet 0: best loss: 446.60582503784303, degree: 0.00016102620275609426, lambda: 1e-10
jet 1: 
New loss: 5108.00889175473, gamma: 1e-10, lambda: 1e-10
New loss: 5098.63216694286, gamma: 2.2122162910704502e-10, lambda: 1e-10
New loss: 5079.534651545113, gamma: 4.893900918477499e-10, lambda: 1e-10
New loss: 5044.244638368951, gamma: 1.0826367338740564e-09, lambda: 1e-10
New loss: 4990.95863071256, gamma: 2.395026619987491e-09, lambda: 1e-10
New loss: 4935.345306703182, gamma: 5.298316906283702e-09, lambda: 1e-10
New loss: 4894.517106916253, gamma: 1.1721022975334793e-08, lambda: 1e-10
New loss: 4843.674440985561, gamma: 2.592943797404667e-08, lambda: 1e-10
New loss: 4748.641892924933, gamma: 5.736152510448681e-08, lambda: 1e-10
New loss: 4596.371974402499, gamma: 1.2689610031679235e-07, lambda: 1e-10
New loss: 4422.672104453381, gamma: 2.807216203941181e-07, lambda: 1e-10
New loss: 4280.311450890442, gamma: 6.210169418915616e-07, lambda: 1e-10
New loss: 4164.365690538143, gamma: 1.373823795

In [6]:
y_tr, tX_tr, indexes_tr, means_tr, std_tr = preprocessing(x_tr_tot, y_tr_tot)
y_te, tX_te, indexes_te, means_te, std_te = preprocessing(x_te_tot, y_te_tot)

In [27]:
list_loss = []
list_w = []
for jet in range(0,8):
    initial_w = np.zeros(tX_tr[jet].shape[1])
    w, loss = reg_logistic_regression(y_tr[jet], tX_tr[jet], 1e-10, initial_w, 100000, gammas_star[jet])
    print(loss)
    list_loss.append(loss)
    list_w.append(w)

3680.055481504468
27764.72327117549
1459.7004266660938
31849.445585489553
648.2671732761505
21567.588545606493
255.56508887001226
8853.572060939954


In [28]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.75582


In [23]:
lambdas_star

[1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10]

In [30]:
gammas_star

[0.00016102620275609426,
 6.723357536499335e-06,
 0.00016102620275609426,
 6.723357536499335e-06,
 0.0003562247890262444,
 6.723357536499335e-06,
 0.0007880462815669921,
 1.4873521072935119e-05]

In [20]:
DATA_TEST_PATH = 'test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_new, indexes_test_new, means_test, stds_test = preprocessing(tX_test) #same function as train,

In [29]:
#cell for logistic
y_pred_test = build_predictions(tX_test_new, indexes_test_new, list_w, logistic=True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]

In [ ]:
[2.848035868435799e-05,
 3.5111917342151275e-06,
 0.0002310129700083158,
 3.5111917342151275e-06,
 0.0002310129700083158,
 3.5111917342151275e-06,
 0.0002310129700083158,
 2.848035868435799e-05]